# Multivariante del método K-Nearest Neighbors

Hasta ahora exploramos como usar un modelo de ML, el KNN simple que uso solo una caracteristica para las predicciones. Si bien esto ayudo a familiarizarse con los conceptos basicos del ML, esta claro que usar una sola caracteristica no refleja la realidad.<br>

Hay 2 formas en las que podemos retocar este modelo para tratar de mejorar la precision (mejorar el RMSE):<br>
- Incrementar el numero de atributos (**caracteristicas**) que el modelo usa para calcular similitudes cuando ranque los vecinos mas cercanos.
- Incrementar **k**, el numero de vecinos cercanos que el modelo usa.

### Eliminacion de caracteristicas no utiles y _**Normalizacion**_ de los datos.

Al seleccionar mas atributos, tenemos que tener cuidado con las columnas que no funcionan bien con la ecuacion de distancia:
- Valores no numericos (ejemplo, ciudad o estado)
    - La ecuacion de distancia euclidiana espera valores numericos.
- Valores perdidos
    - La ecuacion de la distancia euclideana espera un valor por cada observacion y atributo.
- Valores no ordinales (por ejemplo, latitud o longitud)
    - La clasifiacion por distancia euclideana no tiene sentido si todos los valores no son ordinales. 

In [2]:
import pandas as pd
import numpy as np

# Leyendo datos y acomodandolos aleatoreamente
dc_listings = pd.read_csv('dc_airbnb.csv')
dc_listings = dc_listings.loc[ np.random.permutation(len(dc_listings)) ]

# Convirtiendo la columna price a float (limpiando datos)
stripped_commas = dc_listings['price'].str.replace(',', '')
stripped_dollars = stripped_commas.str.replace('$', '')
dc_listings['price'] = stripped_dollars.astype('float')

dc_listings.head()

,host_response_rate,host_acceptance_rate,host_listings_count,accommodates,room_type,bedrooms,bathrooms,beds,price,cleaning_fee,security_deposit,minimum_nights,maximum_nights,number_of_reviews,latitude,longitude,city,zipcode,state
2620,NaN,NaN,1,4,Entire home/apt,2.0,1.5,2.0,125.0,$50.00,$500.00,5,100,2,38.914144,-77.008797,Washington,20002,DC
2063,100%,88%,6,2,Private room,1.0,1.5,1.0,59.0,NaN,$150.00,6,1125,1,38.897817,-76.986683,Washington,20002,DC
1315,81%,72%,3,8,Entire home/apt,2.0,1.0,3.0,250.0,NaN,NaN,1,1125,1,38.915065,-77.019446,Washington,20001,DC
3510,90%,100%,1,3,Entire home/apt,1.0,1.0,1.0,160.0,$50.00,NaN,1,1125,16,38.888043,-76.992011,Washington,20003,DC
2359,100%,75%,2,7,Entire home/apt,3.0,2.0,3.0,250.0,$10.00,$100.00,2,1125,1,38.918137,-77.046417,Washington,20009,DC


In [3]:
print(dc_listings.info())

<class 'pandas.core.frame.DataFrame'>
Index: 3723 entries, 2620 to 1302
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   host_response_rate    3289 non-null   object 
 1   host_acceptance_rate  3109 non-null   object 
 2   host_listings_count   3723 non-null   int64  
 3   accommodates          3723 non-null   int64  
 4   room_type             3723 non-null   object 
 5   bedrooms              3702 non-null   float64
 6   bathrooms             3696 non-null   float64
 7   beds                  3712 non-null   float64
 8   price                 3723 non-null   float64
 9   cleaning_fee          2335 non-null   object 
 10  security_deposit      1426 non-null   object 
 11  minimum_nights        3723 non-null   int64  
 12  maximum_nights        3723 non-null   int64  
 13  number_of_reviews     3723 non-null   int64  
 14  latitude              3723 non-null   float64
 15  longitude             3